In [1]:
import glob
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt


## Загружаем боксы

In [2]:
filelist = glob.glob('images/*.png')
data = [[np.array(Image.open(fname).resize((28, 28)).convert('L')).ravel(), int(fname.split('/')[1].split('.')[0])] for fname in filelist]

# labels = np.array(int(fname.split('/')[1].split('.')[0]) for fname in filelist)


## Ресайз изображений

In [37]:
# new_size = ()

# data[0][0]

In [54]:
pd_data = pd.DataFrame(data, columns=['image', 'label'])
pd_data.rename(columns={"label": "boxid"}, inplace=True)
pd_data = pd_data.astype({'boxid': 'int64'})

In [55]:
# pd_data

In [56]:
target = pd.read_csv('./images_labelling.csv')
target = target.astype({'boxid': 'int64'})

## Мержим колонки

In [57]:
all_data = pd.merge(pd_data, target, on='boxid', how='inner')

In [58]:
# all_data

In [59]:
# train_images = np.array(all_data['image'])
train_images =np.reshape(np.array(list(map(lambda img: np.reshape(img, (1, 784)), all_data['image']))), (3750,784))
train_target = np.array(all_data['label'])

In [60]:
# new_data = np.array(list(map(lambda img: np.reshape(img, (1, 784)), all_data['image'])))
# final = 
# final.shape

### Изучаем данные

In [61]:
# all_data['label'].value_counts()

## Препроцессинг

In [62]:
X_train, X_test, y_train, y_test = train_test_split(train_images, train_target, test_size=0.2, random_state=42)
y_train = np.reshape(y_train.astype(int), (3000, 1))
y_test = np.reshape(y_test.astype(int), (750, 1))
batch_size =len(X_train)

print(X_train.shape, y_train.shape, y_test.shape )

(3000, 784) (3000, 1) (750, 1)


In [63]:
# print(f'type={type(X_train)}')
# print(f'shape={X_train.shape}')
# X_train_reshaped = np.reshape(X_train, (3000, 784))

In [64]:
scaler = MinMaxScaler()
# Train
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
# test
X_test_scaled = scaler.fit_transform(X_test.astype(np.float64))
feature_columns = [tf.feature_column.numeric_column('x', shape=X_train_scaled.shape[1:])]
X_train_scaled.shape[1:]

(784,)

## Строим CNN

In [66]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=36,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 36])
  dense = tf.layers.dense(inputs=pool2_flat, units=7 * 7 * 36, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=25)
  print(logits)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics Evaluation mode
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [67]:
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="train/mnist_convnet_model")

tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'train/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x111d127b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [68]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train_scaled},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

mnist_classifier.train(
    input_fn=train_input_fn,
    steps=16000,
    hooks=[logging_hook]
)



INFO:tensorflow:Calling model_fn.
Tensor("dense_1/BiasAdd:0", shape=(100, 25), dtype=float64)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into train/mnist_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.03853391 0.03712416 0.03950143 ... 0.03730933 0.03552312 0.03916558]
 [0.04082821 0.03778776 0.04252597 ... 0.0385475  0.03511583 0.03839587]
 [0.03389361 0.03710553 0.04380228 ... 0.03740202 0.03668085 0.0391405 ]
 ...
 [0.03547625 0.0338894  0.04003988 ... 0.03682766 0.03694133 0.04091042]
 [0.03861617 0.03714374 0.04370834 ... 0.03782767 0.03437984 0.03758212]
 [0.03693434 0.03679974 0.0420957  ... 0.03814252 0.0389725  0.04026057]]
INFO:tensorflow:loss = 3.217526435852051, step = 1
INFO:tensorflow:probabilities = [[0.03637209 0.03671942 0.04143915 ... 0.03981236 0.0351326 

INFO:tensorflow:loss = 3.205230951309204, step = 701 (40.309 sec)
INFO:tensorflow:probabilities = [[0.03454822 0.03293593 0.04060187 ... 0.03387356 0.03884541 0.03619389]
 [0.04100433 0.0371633  0.04391724 ... 0.03730843 0.03715859 0.03579431]
 [0.03579085 0.03631664 0.04352158 ... 0.03989073 0.03911017 0.03914284]
 ...
 [0.03774698 0.03609574 0.04124442 ... 0.03945389 0.04013838 0.03615463]
 [0.03451573 0.03091001 0.04079397 ... 0.03156475 0.03533281 0.03409274]
 [0.03392621 0.03648674 0.04243562 ... 0.03875679 0.03970052 0.03673668]] (18.905 sec)
INFO:tensorflow:global_step/sec: 2.72449
INFO:tensorflow:probabilities = [[0.03481456 0.03752529 0.04550151 ... 0.03680553 0.04198212 0.03255238]
 [0.03790163 0.03891167 0.03905986 ... 0.03815066 0.04181281 0.03643977]
 [0.03088752 0.03478999 0.04146829 ... 0.04182797 0.03817081 0.03694855]
 ...
 [0.03629171 0.03506671 0.03879633 ... 0.03498826 0.03716233 0.03677394]
 [0.03153031 0.03388199 0.04173164 ... 0.03645963 0.03865223 0.0391119 ]
 [

INFO:tensorflow:global_step/sec: 0.339614
INFO:tensorflow:probabilities = [[0.03732572 0.03867621 0.03603853 ... 0.03364133 0.03869157 0.03519404]
 [0.04045876 0.03758509 0.04443402 ... 0.03867532 0.0376416  0.03257827]
 [0.03380484 0.03501603 0.03668423 ... 0.03598755 0.04243027 0.03465795]
 ...
 [0.03054404 0.03189051 0.03602425 ... 0.03361608 0.03572758 0.03425771]
 [0.03487813 0.03432475 0.03816988 ... 0.03577021 0.0416926  0.03109487]
 [0.03635704 0.03503158 0.03743944 ... 0.034151   0.03883667 0.03277562]] (277.224 sec)
INFO:tensorflow:loss = 3.1932311058044434, step = 1501 (294.453 sec)
INFO:tensorflow:probabilities = [[0.03732478 0.03556162 0.04237778 ... 0.03560045 0.04724096 0.03122471]
 [0.02899313 0.03007712 0.03737199 ... 0.03911966 0.03691237 0.03584888]
 [0.03216518 0.03447628 0.0346278  ... 0.04070286 0.03459276 0.03411316]
 ...
 [0.03015494 0.03668085 0.03896042 ... 0.04174599 0.03459958 0.03420518]
 [0.0361891  0.03740408 0.03954095 ... 0.04379277 0.04001156 0.0363911

INFO:tensorflow:probabilities = [[0.02670544 0.03251035 0.03029722 ... 0.03095107 0.0343889  0.02158109]
 [0.03381642 0.03757745 0.03512627 ... 0.03296853 0.04603714 0.02947698]
 [0.0293055  0.03711597 0.03623374 ... 0.03154702 0.0438062  0.03959778]
 ...
 [0.03284009 0.03270337 0.04009818 ... 0.0448848  0.04068853 0.03961498]
 [0.03855979 0.03492868 0.03349884 ... 0.03848315 0.04025429 0.03374289]
 [0.02778929 0.02862322 0.02762522 ... 0.02739517 0.03962324 0.03104968]] (25.881 sec)
INFO:tensorflow:global_step/sec: 2.21006
INFO:tensorflow:probabilities = [[0.03074093 0.03363562 0.03493077 ... 0.0394365  0.03160362 0.03007586]
 [0.03753395 0.03934466 0.03530918 ... 0.03244227 0.03859178 0.03091271]
 [0.03302689 0.0327233  0.03707586 ... 0.02807496 0.03465854 0.03552732]
 ...
 [0.04511502 0.03582413 0.0386867  ... 0.0419114  0.04131198 0.03458213]
 [0.03064041 0.02596272 0.03008083 ... 0.03345397 0.03617691 0.02888777]
 [0.02266315 0.02369951 0.02734642 ... 0.03025005 0.03318877 0.03649

INFO:tensorflow:probabilities = [[0.01975885 0.02706492 0.02282935 ... 0.02681917 0.02285328 0.02726192]
 [0.01471474 0.02543039 0.03356602 ... 0.02503953 0.03139249 0.01808412]
 [0.03805463 0.02782231 0.04095538 ... 0.04292066 0.03946265 0.03468961]
 ...
 [0.04491468 0.05766796 0.04591552 ... 0.05705638 0.04530422 0.03427761]
 [0.0540642  0.04630793 0.03072259 ... 0.03737244 0.06848454 0.02627416]
 [0.01184649 0.02206778 0.02306113 ... 0.02962301 0.0284363  0.02802731]] (17.622 sec)
INFO:tensorflow:loss = 3.045746088027954, step = 3001 (35.636 sec)
INFO:tensorflow:probabilities = [[0.03830121 0.04417176 0.03095629 ... 0.03075829 0.04857421 0.03726528]
 [0.01965935 0.02243296 0.02746479 ... 0.02408177 0.03219055 0.02325357]
 [0.04095229 0.05295722 0.04483487 ... 0.03365297 0.03676792 0.03045212]
 ...
 [0.03795653 0.02614103 0.04282605 ... 0.03718887 0.04199102 0.02629757]
 [0.01456159 0.02673334 0.02371363 ... 0.01474991 0.02130307 0.0200155 ]
 [0.0400215  0.04317265 0.02912164 ... 0.0

...

INFO:tensorflow:loss = 1.7309105396270752, step = 12901 (40.898 sec)
INFO:tensorflow:probabilities = [[0.01192826 0.12191973 0.26266995 ... 0.09891174 0.01732    0.00348428]
 [0.02644692 0.12024331 0.19629454 ... 0.19585297 0.03920221 0.00841673]
 [0.00000002 0.00014112 0.000051   ... 0.00000748 0.00005147 0.00217102]
 ...
 [0.00030073 0.19151812 0.21407121 ... 0.06802722 0.17018248 0.04820242]
 [0.         0.00000164 0.00000003 ... 0.00000001 0.00000023 0.00326519]
 [0.11825561 0.05287024 0.00869303 ... 0.00973771 0.00400744 0.00691601]] (20.787 sec)
INFO:tensorflow:global_step/sec: 2.58294
INFO:tensorflow:probabilities = [[0.02017972 0.07248651 0.04776464 ... 0.1875818  0.09951936 0.00675297]
 [0.00000826 0.00006518 0.00005609 ... 0.00035168 0.00497928 0.00124609]
 [0.00000054 0.00042308 0.00001521 ... 0.00003413 0.00024491 0.0006136 ]
 ...
 [0.         0.00003495 0.00001107 ... 0.00000594 0.0000017  0.000246  ]
 [0.00000006 0.00018111 0.00004697 ... 0.00001471 0.00001011 0.00027396]

INFO:tensorflow:probabilities = [[0.         0.00000577 0.00000007 ... 0.00000003 0.00000096 0.00057601]
 [0.03901671 0.13323139 0.13970641 ... 0.10532353 0.09755963 0.01659779]
 [0.         0.00000048 0.00000007 ... 0.         0.0000001  0.00644439]
 ...
 [0.2029978  0.04322541 0.00232562 ... 0.0125418  0.01215448 0.07625914]
 [0.00000081 0.00028807 0.00011441 ... 0.00003783 0.00003892 0.00193048]
 [0.         0.00003337 0.00000337 ... 0.00000083 0.00003742 0.00587021]] (18.259 sec)
INFO:tensorflow:loss = 1.6019256114959717, step = 13701 (38.302 sec)
INFO:tensorflow:probabilities = [[0.04219946 0.02678814 0.09144847 ... 0.14843853 0.04736522 0.10080383]
 [0.03674315 0.20682782 0.27593559 ... 0.09770516 0.04137788 0.00294677]
 [0.         0.00000244 0.00000004 ... 0.00000037 0.00000699 0.00104827]
 ...
 [0.00000001 0.00021376 0.00001177 ... 0.00000504 0.00055205 0.00056881]
 [0.00024655 0.00007775 0.00000296 ... 0.00001085 0.00000633 0.98753991]
 [0.00000002 0.00017653 0.00000293 ... 0

INFO:tensorflow:probabilities = [[0.00066435 0.04661389 0.01127255 ... 0.01982339 0.4850535  0.06564143]
 [0.00000008 0.00007983 0.00001193 ... 0.00000215 0.00003647 0.00095937]
 [0.74128211 0.01504038 0.00859448 ... 0.02735407 0.00135508 0.00304079]
 ...
 [0.0141234  0.0610499  0.00577496 ... 0.06585497 0.05277961 0.01095171]
 [0.03347107 0.04958762 0.01752281 ... 0.11072015 0.05677492 0.00175424]
 [0.         0.00006177 0.00000054 ... 0.00000008 0.00000694 0.00077672]] (24.620 sec)
INFO:tensorflow:global_step/sec: 2.16994
INFO:tensorflow:probabilities = [[0.00000288 0.00000027 0.00000004 ... 0.00000008 0.00000036 0.00007542]
 [0.         0.00000002 0.00000243 ... 0.00000007 0.00000002 0.97465095]
 [0.02963114 0.03929648 0.01313907 ... 0.03795884 0.06631068 0.0058473 ]
 ...
 [0.00000074 0.00069882 0.00011544 ... 0.0000469  0.00013716 0.00103806]
 [0.02027628 0.15402562 0.02607306 ... 0.05376787 0.02636057 0.01090779]
 [0.00000494 0.00037332 0.00008716 ... 0.00003204 0.00004483 0.03461

INFO:tensorflow:loss = 1.6209356784820557, step = 15201 (40.496 sec)
INFO:tensorflow:Saving checkpoints for 15238 into train/mnist_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.68647823 0.0078064  0.00552417 ... 0.01309097 0.0107357  0.00140431]
 [0.06958229 0.00604915 0.04739431 ... 0.10723188 0.0242083  0.03150499]
 [0.00080493 0.027015   0.01508883 ... 0.04198736 0.64924978 0.01077779]
 ...
 [0.0163278  0.05313492 0.01223625 ... 0.04205022 0.02242153 0.00590859]
 [0.33962074 0.04443847 0.02442482 ... 0.13651248 0.01641894 0.00434821]
 [0.02307002 0.00556975 0.00439569 ... 0.0507242  0.00800814 0.01308307]] (24.565 sec)
INFO:tensorflow:global_step/sec: 2.1508
INFO:tensorflow:probabilities = [[0.00872532 0.03380228 0.14779218 ... 0.18204639 0.03708659 0.00208677]
 [0.04810095 0.13046247 0.03499512 ... 0.01472264 0.06891796 0.0631535 ]
 [0.00000248 0.00075217 0.00004401 ... 0.00005255 0.00005019 0.01772704]
 ...
 [0.06332045 0.05024275 0.03721564 ... 0.02431276 0.067953

INFO:tensorflow:Saving checkpoints for 16000 into train/mnist_convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 1.2883155345916748.


In [69]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test_scaled},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
Tensor("dense_1/BiasAdd:0", shape=(?, 25), dtype=float64)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-11-10T11:53:36Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/mnist_convnet_model/model.ckpt-16000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-11-10-11:53:37
INFO:tensorflow:Saving dict for global step 16000: accuracy = 0.46266666, global_step = 16000, loss = 1.5904447
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 16000: train/mnist_convnet_model/model.ckpt-16000
{'accuracy': 0.46266666, 'loss': 1.5904447, 'global_step': 16000}
